In [6]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [11]:
def initialize_driver(executable_path):

    s = Service(executable_path=executable_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

In [12]:
def scrape_reviews(driver, url):

    driver.get(url)
    time.sleep(3)

    # Click on the reviews button
    reviews_button = driver.find_element(By.LINK_TEXT,'See All Customer Reviews')


    reviews_button.click()
    time.sleep(3)

    # Get total number of reviews
    tot_reviews = driver.find_element(By.XPATH, '/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text
    match = re.search(r'\(([\d,]+)\sReviews\)', tot_reviews)
    if match:
        review_count_str = match.group(1)
        # Remove commas from the string and convert to integer
        review_count = int(review_count_str.replace(',', ''))
        print("Number of reviews:", review_count)
    else:
        print("Number of reviews not found")
    n = round(review_count/20)

    star_rating = []
    review = []
    name = []
    review_title = []
    votes = []

    for j in range(n):
        try:
            review_elements = driver.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
            total_reviews = len(review_elements)

            for i in range(1, total_reviews + 1):
                try:
                    star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
                    rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/p').text
                    rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
                    title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
                    helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]
                except Exception as e:
                    print(e)

                star_rating.append(star)
                review.append(rev)
                name.append(rev_name)
                review_title.append(title)
                votes.append(helpful_votes)
            
            # Click the button to navigate to the next page
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
            )
            next_button.click()
            time.sleep(3)
        except NoSuchElementException:
            break

    return star_rating, review, name, review_title, votes

In [13]:
def main():
    urls = ['https://www.bestbuy.com/site/mcafee-total-protection-5-device-antivirus-internet-security-software-1-year-subscription-android-apple-ios-chrome-mac-os-windows-digital/6517326.p?skuId=6517326',
            'https://www.bestbuy.com/site/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690.p?skuId=6346690',
            'https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535',
            'https://www.bestbuy.com/site/apple-iphone-13-5g-128gb-unlocked-midnight/6417788.p?skuId=6417788',
            'https://www.bestbuy.com/site/samsung-galaxy-s23-ultra-256gb-unlocked-phantom-black/6529723.p?skuId=6529723']

    driver = initialize_driver('D:\CU Boulder - Data Science\Spring 2024\Data Mining\chromedriver.exe')

    all_dfs = []

    for url in urls:
        star_rating, review, name, review_title, votes = scrape_reviews(driver, url)
        df = pd.DataFrame({
            'Star Rating': star_rating,
            'Review': review,
            'Reviewer Name': name,
            'Review Title': review_title,
            'Helpful Votes': votes
        })
        all_dfs.append(df)

    driver.quit()

    return all_dfs

In [14]:
# Call the main function to get data for all URLs
all_data_dfs = main()

Number of reviews: 2041
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[1]/div/div[2]/div[5]/p"}
  (Session info: chrome=122.0.6261.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F6C05E42+3538674]
	(No symbol) [0x00007FF6F6824C02]
	(No symbol) [0x00007FF6F66D5AEB]
	(No symbol) [0x00007FF6F671BF4E]
	(No symbol) [0x00007FF6F671C0CC]
	(No symbol) [0x00007FF6F675E477]
	(No symbol) [0x00007FF6F673F0EF]
	(No symbol) [0x00007FF6F675BDE3]
	(No symbol) [0x00007FF6F673EE53]
	(No symbol) [0x00007FF6F670F514]
	(No symbol) [0x00007FF6F6710631]
	GetHandleVerifier [0x00007FF6F6C36CAD+3738973]
	GetHandleVerifier [0x00007FF6F6C8C506+4089270]
	GetHandleVerifier [0x00007FF6F6C84823+4057299]
	GetHandleVerifier [0x00007FF6F6955C49+720121]
	(No symbol) [0x000

In [15]:
all_data_dfs

[     Star Rating                                             Review  \
 0              5  Excellent phone. Great ability to multitask, f...   
 1              5  Got the Samsung Galaxy S23 Ultra as a late bir...   
 2              5  The Samsung Galaxy S23 Ultra is a phone that a...   
 3              5  This phone is absolutely amazingly.  This phon...   
 4              5  After over 5 years with iPhones, I took a chan...   
 ...          ...                                                ...   
 2035           5  The best phone EVER!!\nThe Camera:\nAmazing it...   
 2036           1  The best phone EVER!!\nThe Camera:\nAmazing it...   
 2037           3  The best phone EVER!!\nThe Camera:\nAmazing it...   
 2038           1  The best phone EVER!!\nThe Camera:\nAmazing it...   
 2039           2  The best phone EVER!!\nThe Camera:\nAmazing it...   
 
      Reviewer Name                                Review Title Helpful Votes  
 0         Salvador                             Excell

In [16]:
# Save each DataFrame as a CSV file
for i, df in enumerate(all_data_dfs):
    df.to_csv(f'url_{i+1}_data.csv', index=False)